## Install master branch of TensorTrade

In [0]:
!pip install git+https://github.com/tensortrade-org/tensortrade.git -U

In [3]:
import tensortrade.env.default as default

from tensortrade.feed.core import Stream, DataFeed
from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.oms.wallets import Portfolio, Wallet
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC, ETH, LTC

## Load Data for Exchanges

Using the `tensortrade.data.cdd` module you can load data from any `csv` file provided at:
- https://www.cryptodatadownload.com/data/northamerican/

using the `CryptoDataDownload` class.

In [4]:
cdd = CryptoDataDownload()
bitfinex_btc = cdd.fetch("Bitfinex", "USD", "BTC", "1h")
bitfinex_eth = cdd.fetch("Bitfinex", "USD", "ETH", "1h")

bitstamp_btc = cdd.fetch("Bitstamp", "USD", "BTC", "1h")
bitstamp_eth = cdd.fetch("Bitstamp", "USD", "ETH", "1h")
bitstamp_ltc = cdd.fetch("Bitstamp", "USD", "LTC", "1h")

## Inspect Transactions of `SimpleOrders`

In [5]:
bitfinex = Exchange("bitfinex", service=execute_order)(
    Stream.source(list(bitfinex_btc['close'][-100:]), dtype="float").rename("USD-BTC"),
    Stream.source(list(bitfinex_eth['close'][-100:]), dtype="float").rename("USD-ETH")
)

bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(bitstamp_btc['close'][-100:]), dtype="float").rename("USD-BTC"),
    Stream.source(list(bitstamp_eth['close'][-100:]), dtype="float").rename("USD-ETH"),
    Stream.source(list(bitstamp_ltc['close'][-100:]), dtype="float").rename("USD-LTC")
)

portfolio = Portfolio(USD, [
    Wallet(bitfinex, 10000 * USD),
    Wallet(bitfinex, 10 * BTC),
    Wallet(bitfinex, 5 * ETH),
    Wallet(bitstamp, 1000 * USD),
    Wallet(bitstamp, 5 * BTC),
    Wallet(bitstamp, 20 * ETH),
    Wallet(bitstamp, 3 * LTC)
])

feed = DataFeed([
    Stream.source(list(bitstamp_eth['volume'][-100:]), dtype="float").rename("volume:/USD-ETH"),
    Stream.source(list(bitstamp_ltc['volume'][-100:]), dtype="float").rename("volume:/USD-LTC")
])

env = default.create(
    portfolio=portfolio,
    action_scheme=default.actions.SimpleOrders(),
    reward_scheme=default.rewards.SimpleProfit(),
    feed=feed
)

done = False
obs = env.reset()
while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)

portfolio.ledger.as_frame().head(7)

,poid,step,source,target,memo,amount,free,locked,locked_poid
0,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp:USD/free,bitstamp:USD/locked,LOCK (LOCK FOR ORDER),1000.00 USD,0.00 USD,1000.00 USD,1000.00 USD
1,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp:USD/locked,bitstamp,WITHDRAWAL (COMMISSION),3.00 USD,0.00 USD,997.00 USD,997.00 USD
2,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp:USD/locked,bitstamp,WITHDRAWAL (FILL ORDER),997.00 USD,0.00 USD,0.00 USD,0.00 USD
3,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp,bitstamp:BTC/locked,DEPOSIT (TRADED 997.00 USD bitstamp:USD/BTC @ ...,0.10940979 BTC,5.00000000 BTC,0.10940979 BTC,0.10940979 BTC
4,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp:USD/locked,bitstamp:USD/free,UNLOCK USD (COMPLETED),0.00 USD,0.00 USD,0.00 USD,0.00 USD
5,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp:BTC/locked,bitstamp:BTC/free,UNLOCK BTC (COMPLETED),0.10940979 BTC,5.10940979 BTC,0.00000000 BTC,0.00000000 BTC
6,9ba20790-6b44-4f65-8cc3-545528dda301,2,bitfinex:BTC/free,bitfinex:BTC/locked,LOCK (LOCK FOR ORDER),3.00000000 BTC,7.00000000 BTC,3.00000000 BTC,3.00000000 BTC


## Inspect Transactions of `ManagedRiskOrders`

In [6]:
portfolio = Portfolio(USD, [
    Wallet(bitfinex, 10000 * USD),
    Wallet(bitfinex, 0 * BTC),
    Wallet(bitfinex, 0 * ETH),
])

env = default.create(
    portfolio=portfolio,
    action_scheme=default.actions.ManagedRiskOrders(),
    reward_scheme=default.rewards.SimpleProfit(),
    feed=feed
)

done = False

while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)

portfolio.ledger.as_frame().head(20)

,poid,step,source,target,memo,amount,free,locked,locked_poid
0,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp:USD/free,bitstamp:USD/locked,LOCK (LOCK FOR ORDER),1000.00 USD,0.00 USD,1000.00 USD,1000.00 USD
1,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp:USD/locked,bitstamp,WITHDRAWAL (COMMISSION),3.00 USD,0.00 USD,997.00 USD,997.00 USD
2,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp:USD/locked,bitstamp,WITHDRAWAL (FILL ORDER),997.00 USD,0.00 USD,0.00 USD,0.00 USD
3,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp,bitstamp:BTC/locked,DEPOSIT (TRADED 997.00 USD bitstamp:USD/BTC @ ...,0.10940979 BTC,5.00000000 BTC,0.10940979 BTC,0.10940979 BTC
4,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp:USD/locked,bitstamp:USD/free,UNLOCK USD (COMPLETED),0.00 USD,0.00 USD,0.00 USD,0.00 USD
5,ba218138-98c9-4e2c-b2fd-c12c79f2a07e,1,bitstamp:BTC/locked,bitstamp:BTC/free,UNLOCK BTC (COMPLETED),0.10940979 BTC,5.10940979 BTC,0.00000000 BTC,0.00000000 BTC
6,9ba20790-6b44-4f65-8cc3-545528dda301,2,bitfinex:BTC/free,bitfinex:BTC/locked,LOCK (LOCK FOR ORDER),3.00000000 BTC,7.00000000 BTC,3.00000000 BTC,3.00000000 BTC
7,9ba20790-6b44-4f65-8cc3-545528dda301,2,bitfinex:BTC/locked,bitfinex,WITHDRAWAL (COMMISSION),0.00900000 BTC,7.00000000 BTC,2.99100000 BTC,2.99100000 BTC
8,9ba20790-6b44-4f65-8cc3-545528dda301,2,bitfinex:BTC/locked,bitfinex,WITHDRAWAL (FILL ORDER),2.99100000 BTC,7.00000000 BTC,0.00000000 BTC,0.00000000 BTC
9,9ba20790-6b44-4f65-8cc3-545528dda301,2,bitfinex,bitfinex:USD/locked,DEPOSIT (TRADED 2.99100000 BTC bitfinex:USD/BT...,27249.98 USD,10000.00 USD,27249.98 USD,27249.98 USD


## Transactions in Spreadsheets

To take a closer look at the transactions that are happening within the system, copy the transactions to a csv file and load it into any spreadsheet software. If where you are running this allows access to the system clipboard, you can directly copy the frame to your system clipboard by doing the following:
- `portfolio.ledger.as_frame().to_clipboard(index=False)`

Then just paste into any spreadsheet software (e.g. Execel, Google Sheets)

image.png